In [1]:
import numpy as np
import pandas as pd
import pyodbc
import pandas as pd
import numpy as np
from pprint import pprint
import datetime
from configparser import ConfigParser


In [2]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [3]:
# Code for Audit Integration
cursor.execute("""insert into FINCORE_DB.Fincore_Analytics.AUDIT_INTGN (JOB_RUN_ID,JOB_NAME,JOB_START_DATE,JOB_STATUS,TARGET_TBL,SOURCE_TBL,SOURCE_MAX_DATE) values(COALESCE((select (max(JOB_RUN_ID)+1) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN),1),'CASE_RISK',GETDATE(),'Error','RISK_CASE','CASE_MASTER',(SELECT MAX(T.MAX_DATE) FROM (select MAX(CREATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore.CASE_MASTER UNION ALL select MAX(UPDATE_DATE) AS MAX_DATE from FINCORE_DB.Fincore.CASE_MASTER )T))""")

job_id = pd.read_sql_query("select max(JOB_RUN_ID) as job_run_id from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN with(nolock) where job_name = 'CASE_RISK' ",conn)
job_id = job_id.iloc[0,0]
conn.commit()

In [4]:
job_id

91

In [5]:
%store job_id

Stored 'job_id' (int64)


In [5]:
incremental_data = pd.read_sql_query("select * from FINCORE_DB.Fincore.Case_Master WHERE COALESCE (Update_Date,create_date) > COALESCE (( select MAX( SOURCE_MAX_DATE ) from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN where JOB_STATUS='Success' and JOB_NAME='CASE_RISK' ),cast('1900-01-01 00:00:00.0000000' as datetime2))",conn)

In [6]:
no_of_records = len(incremental_data)

In [7]:
no_of_records

0

In [8]:
cursor.execute('''MERGE INTO FINCORE_DB.FINCORE_ANALYTICS.RISK_CASE AS TARGET USING(
SELECT A.BATCH_ID,A.REPORT_ID,
A.CASE_MASTER_ID,
B.REPORT_TYPE,
CASE WHEN (B.RISK_SCORE = 0 OR B.RISK_SCORE IS NULL) AND (C.RISK_SCORE = 0 OR C.RISK_SCORE IS NULL) THEN 0
ELSE (SQUARE(COALESCE(B.RISK_SCORE,0))+SQUARE(COALESCE(C.RISK_SCORE,0)))/(COALESCE(B.RISK_SCORE,0)+COALESCE(C.RISK_SCORE,0)) 
END AS RISK_SCORE,
GETDATE() AS CREATE_DATE,
GETDATE() AS UPDATE_DATE 
FROM FINCORE_DB.FINCORE.CASE_MASTER A 
LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_REPORT B ON A.REPORT_ID = B.REPORT_ID 
LEFT JOIN FINCORE_DB.FINCORE_ANALYTICS.RISK_NETWORK C ON C.CASE_MASTER_ID = A.CASE_MASTER_ID
WHERE COALESCE (A.Update_Date,A.create_date) > COALESCE (( select MAX( SOURCE_MAX_DATE )
from FINCORE_DB.Fincore_Analytics.AUDIT_INTGN
where JOB_STATUS='Success' and JOB_NAME='CASE_RISK' )
,cast('1900-01-01 00:00:00.0000000' as datetime2))

)
 AS SOURCE ON TARGET.BATCH_ID=SOURCE.BATCH_ID AND 
 TARGET.REPORT_ID=SOURCE.REPORT_ID
 AND TARGET.CASE_MASTER_ID=SOURCE.CASE_MASTER_ID
 WHEN MATCHED THEN UPDATE SET TARGET.RISK_SCORE=SOURCE.RISK_SCORE,
 TARGET.UPDATE_DATE=SOURCE.UPDATE_DATE 
 WHEN NOT MATCHED BY TARGET THEN INSERT(BATCH_ID,REPORT_ID,CASE_MASTER_ID,REPORT_TYPE,RISK_SCORE,CREATE_DATE,UPDATE_DATE)
 VALUES(SOURCE.BATCH_ID,SOURCE.REPORT_ID,SOURCE.CASE_MASTER_ID,SOURCE.REPORT_TYPE,SOURCE.RISK_SCORE,GETDATE(),GETDATE());''')
conn.commit()




In [9]:
cursor.execute('''UPDATE FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN SET JOB_END_DATE = GETDATE() , JOB_STATUS = 'Success', no_of_record = {}  where job_run_id = {} '''.format(no_of_records,job_id))
conn.commit()
conn.close()

In [ ]:
print("CASE RISK COMPLETED.")